In [3]:
cd /content/drive/My Drive/GSoC2020/audio_visual_emotion/GSoC2020/

/content/drive/My Drive/GSoC2020/audio_visual_emotion/GSoC2020


In [22]:
import numpy as np
import os
import sys

from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers.convolutional import Conv2D
from keras.layers import LSTM, Input, Flatten, Embedding, Convolution1D,Dropout, concatenate
from keras.optimizers import SGD, Adam, RMSprop

from scipy import signal
from sklearn.preprocessing import label_binarize

import pickle
from features import *
from helper import *


## Loading data 

In [9]:
code_path = os.path.dirname(os.path.realpath(os.getcwd()))
emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
#data_path = code_path + "/../data/sessions/"
data_path = "../../Data/IEMOCAP/"
framerate = 16000

In [10]:
# Load IEMOCAP data
with open(data_path + '/../'+'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [13]:
def calculate_features(frames, freq, options):
    window_sec = 0.2
    window_n = int(freq * window_sec)

    st_f = stFeatureExtraction(frames, freq, window_n, window_n / 2)

    if st_f.shape[1] > 2:
        i0 = 1
        i1 = st_f.shape[1] - 1
        if i1 - i0 < 1:
            i1 = i0 + 1
        
        deriv_st_f = np.zeros((st_f.shape[0], i1 - i0), dtype=float)
        for i in range(i0, i1):
            i_left = i - 1
            i_right = i + 1
            deriv_st_f[:st_f.shape[0], i - i0] = st_f[:, i]
        return deriv_st_f
    elif st_f.shape[1] == 2:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f
    else:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f

In [14]:
x_train_speech = []

counter = 0
for ses_mod in data2:
    x_head = ses_mod['signal']
    st_features = calculate_features(x_head, framerate, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=100)
    x_train_speech.append( st_features.T )
    counter+=1
    if(counter%100==0):
        print(counter)
    
x_train_speech = np.array(x_train_speech)
x_train_speech.shape

100
200
300
400
500
600
700
800
900


(950, 100, 34)

In [16]:
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_train_mocap.append( x_rot )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,165,1)
x_train_mocap.shape

(950, 200, 165, 1)

In [17]:
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = label_binarize(Y,emotions_used)

Y.shape

(950, 4)

## Models

In [31]:
model_speech = Sequential()
model_speech.add(Flatten(input_shape=(100, 34)))
model_speech.add(Dense(64))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.3))
model_speech.add(Dense(128))

model_mocap = Sequential()
model_mocap.add(Conv2D(32, 3, strides=(2, 2), border_mode='same', input_shape=(200, 165, 1)))
model_mocap.add(Dropout(0.3))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Flatten())
model_mocap.add(Dense(128))


model_combined = concatenate([model_speech.output, model_mocap.output], axis=-1)
model_combined = Activation('relu')(model_combined)
model_combined = Dense(256, activation='relu')(model_combined)
output = Dense(4, activation="softmax")(model_combined)

model = Model(inputs=[model_speech.input, model_mocap.input], outputs=output)


#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer='Adam' ,metrics=['acc'])

model_speech.summary()
model_mocap.summary()
model.summary()

print("Model1 Built")

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_12 (Flatten)         (None, 3400)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 64)                217664    
_________________________________________________________________
activation_33 (Activation)   (None, 64)                0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 128)               8320      
Total params: 225,984
Trainable params: 225,984
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_15"
_________________________________________________________________
Layer (type)            

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, strides=(2, 2), input_shape=(200, 165,..., padding="same")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, strides=(2, 2), padding="same")`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, strides=(2, 2), padding="same")`
  from ipykernel import kernelapp as app


In [32]:
hist = model.fit([x_train_speech, x_train_mocap], Y, 
                 batch_size=64, nb_epoch=50, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 760 samples, validate on 190 samples
Epoch 1/50
760/760 [==============================] - 1s 1ms/step - loss: 11.3570 - acc: 0.3184 - val_loss: 1.6881 - val_acc: 0.3316
Epoch 2/50
760/760 [==============================] - 1s 671us/step - loss: 1.3683 - acc: 0.4434 - val_loss: 1.3915 - val_acc: 0.3421
Epoch 3/50
760/760 [==============================] - 1s 671us/step - loss: 1.2471 - acc: 0.4724 - val_loss: 1.3110 - val_acc: 0.3895
Epoch 4/50
760/760 [==============================] - 1s 675us/step - loss: 1.1740 - acc: 0.5171 - val_loss: 1.2831 - val_acc: 0.3842
Epoch 5/50
760/760 [==============================] - 1s 673us/step - loss: 1.1141 - acc: 0.5092 - val_loss: 1.1666 - val_acc: 0.4474
Epoch 6/50
760/760 [==============================] - 1s 684us/step - loss: 1.0742 - acc: 0.5382 - val_loss: 1.1558 - val_acc: 0.4579
Epoch 7/50
760/760 [==============================] - 0s 651us/step - loss: 1.0266 - acc: 0.5500 - val_loss: 1.1860 - val_acc: 0.4105
Epoch 8/50
760/76

In [33]:
# This model is basically the combination of audio-only and visua-only model

model_speech = Sequential()
model_speech.add(Convolution1D(32, 3, border_mode='same', input_shape=(100, 34)))
model_speech.add(Dropout(0.2))
model_speech.add(Activation('relu'))
model_speech.add(Convolution1D(32, 3, border_mode='same'))
model_speech.add(Dropout(0.2))
model_speech.add(Activation('relu'))
model_speech.add(Flatten())
model_speech.add(Dropout(0.2))
model_speech.add(Dense(128))


model_mocap = Sequential()
model_mocap.add(Conv2D(32, 3, strides=(2, 2), border_mode='same', input_shape=(200, 165, 1)))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Flatten())
model_mocap.add(Dense(128))

model_combined = concatenate([model_speech.output, model_mocap.output], axis=-1)
model_combined = Activation('relu')(model_combined)
model_combined = Dense(256, activation='relu')(model_combined)
output = Dense(4, activation="softmax")(model_combined)

model = Model(inputs=[model_speech.input, model_mocap.input], outputs=output)


#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer='Adam' ,metrics=['acc'])

model_speech.summary()
model_mocap.summary()
model.summary()

print("Model1 Built")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, input_shape=(100, 34), padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, padding="same")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, strides=(2, 2), input_shape=(200, 165,..., padding="same")`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, strides=(2, 2), padding="same")

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 100, 32)           3296      
_________________________________________________________________
dropout_34 (Dropout)         (None, 100, 32)           0         
_________________________________________________________________
activation_38 (Activation)   (None, 100, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 100, 32)           3104      
_________________________________________________________________
dropout_35 (Dropout)         (None, 100, 32)           0         
_________________________________________________________________
activation_39 (Activation)   (None, 100, 32)           0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 3200)            

In [34]:
hist = model.fit([x_train_speech, x_train_mocap], Y, 
                 batch_size=64, nb_epoch=50, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 760 samples, validate on 190 samples
Epoch 1/50
760/760 [==============================] - 1s 1ms/step - loss: 7.3043 - acc: 0.3303 - val_loss: 1.5679 - val_acc: 0.2579
Epoch 2/50
760/760 [==============================] - 0s 635us/step - loss: 1.3662 - acc: 0.4132 - val_loss: 1.3821 - val_acc: 0.2842
Epoch 3/50
760/760 [==============================] - 0s 633us/step - loss: 1.2966 - acc: 0.4461 - val_loss: 1.3682 - val_acc: 0.2737
Epoch 4/50
760/760 [==============================] - 0s 611us/step - loss: 1.2646 - acc: 0.4513 - val_loss: 1.3446 - val_acc: 0.3211
Epoch 5/50
760/760 [==============================] - 0s 655us/step - loss: 1.2472 - acc: 0.4566 - val_loss: 1.3460 - val_acc: 0.3105
Epoch 6/50
760/760 [==============================] - 0s 618us/step - loss: 1.2427 - acc: 0.4579 - val_loss: 1.3528 - val_acc: 0.2842
Epoch 7/50
760/760 [==============================] - 0s 625us/step - loss: 1.2423 - acc: 0.4632 - val_loss: 1.3456 - val_acc: 0.3263
Epoch 8/50
760/760

In [ ]:
!git status